In [ ]:
# This block allows us to import from the benchmark folder,
# as if it was a package installed using pip
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [2]:
import torch
from util import datasets, methods, models
from os import path
from benchmark.impact_coverage import impact_coverage
import itertools

In [3]:
device = "cuda"
dataset_name = "CIFAR10"
model_name = "resnet20"
download_dataset = False
batch_size = 8
n_batches = 32
target_label = 0
data_root = "../data"
normalize_attrs = True
patch_size_percent = 0.1

In [4]:
patch = torch.load(path.join(data_root, "patches", f"{dataset_name}_{model_name}_patch.pt"))
dataset = datasets.Cifar(batch_size=batch_size, data_location=path.join(data_root, dataset_name))
model = models.CifarResnet(version=model_name, params_loc=path.join(data_root, f"models/{dataset_name}_{model_name}.pth"),
                           output_logits=True)
model.to(device)
model.eval()
pass  # Prevents output from model.eval()

In [5]:
attribution_methods = {
    "GuidedGradCAM": methods.GuidedGradCAM(model, model.get_last_conv_layer(), normalize=normalize_attrs),
    "Gradient": methods.Gradient(model, normalize=normalize_attrs),
    "InputXGradient": methods.InputXGradient(model, normalize=normalize_attrs),
    "IntegratedGradients": methods.IntegratedGradients(model, normalize=normalize_attrs),
    "GuidedBackprop": methods.GuidedBackprop(model, normalize=normalize_attrs),
    "Deconvolution": methods.Deconvolution(model, normalize=normalize_attrs),
    "Random": methods.Random(normalize=normalize_attrs)
}

In [6]:
impact_coverage(data=itertools.islice(dataset.get_dataloader(train=False), n_batches), sample_shape=dataset.sample_shape, patch=patch, patch_size_percent=0.1, model=model, methods=attribution_methods,
                device=device, target_label=0)

/home/arne/Documents/PhD/projects/benchmark/benchmark/impact_coverage/impact_coverage.py:17: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  samples, labels = torch.tensor(samples), torch.tensor(labels)
/home/arne/Documents/PhD/projects/benchmark/venv/lib/python3.6/site-packages/captum/attr/_core/guided_backprop_deconvnet.py:56: UserWarning: Setting backward hooks on ReLU activations.The hooks will be removed after the attribution is finished
  "Setting backward hooks on ReLU activations."


{'GuidedGradCAM': 0.30138679732979706,
 'Gradient': 0.004563041102069851,
 'InputXGradient': 0.056338028169014086,
 'IntegratedGradients': 0.16980331545291996,
 'GuidedBackprop': 0.2608198595459742,
 'Deconvolution': 0.049331255210409944,
 'Random': 0.05181888369135747}